# scroll down and click on show widget

# if the simulation doesn't work
####    - refresh
####    -  click again on show widget

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%pip install matplotlib -qqq
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [4]:
import ipywidgets as widgets
from ipywidgets import Layout
from io import BytesIO
import colorsys
import pickle # package use to save data
from IPython.display import display, Image
import nbinteract as nbi

In [5]:
def payoff_matrix_param(alpha, beta, gamma, Alpha, Beta, Gamma):
    A = np.zeros((2,3,3))
    Al = np.matrix([[0, -alpha, -beta],
                   [alpha, 0, -gamma],
                   [beta, gamma, 0]])
    Ah = np.matrix([[0, Alpha, Beta],
                   [-Alpha, 0, Gamma],
                   [-Beta, -Gamma, 0]])
    A[0]=Al
    A[1]=Ah    
    return A    

In [6]:
def F(X, An):
    phi = X.dot(An).dot(X)
#    phi = X.dot(An.dot(X))
    return X*(An.dot(X) - phi)

In [7]:
def plot_determinist(A, show=True):
    nbre_d_eq = 3
#    T = np.linspace(0, 100000, 1000001)
    T = np.linspace(1, 10000, 100001)
    dt = T[1] - T[0]
    XX = np.zeros((len(T), nbre_d_eq))
    X0 = np.zeros(nbre_d_eq) + 1./nbre_d_eq
    
    #plt.subplots(1,2,figsize = (20, 6))
    XX = np.zeros((2, len(T), nbre_d_eq))
    for ind_n in range(2):
    #    plt.figure(figsize = (10, 5))
        XX[ind_n, 0] = X0
        for i, t in enumerate(T[:-1:]):
            An = A[ind_n]
            XX[ind_n, i+1] = XX[ind_n, i] + dt*F(XX[ind_n, i], An)
    
#    plt.figure(figsize = (20, 5))
    for ind_n, n in enumerate(["low", "high"]):   
        if(show):
            plt.figure(figsize = (10, 7))
        else:
            plt.subplot(1,2,ind_n+1)
        for i in range(3):
            plt.semilogx(T, XX[ind_n,:,i], label=Species[i])
       # plt.plot(T, XX[:,3], label="x_4")
        #plt.plot(T, np.sum(XX, axis = 1), "--", color = "grey", label="sum")
        plt.legend(fontsize = 15)
        plt.title("Nitrogen "+n, fontsize=25)
        plt.xlabel("Time (log scale)", fontsize = 20)
        plt.ylabel("biomass proportion", fontsize = 20)
        plt.xticks(fontsize = 15)
        #plt.savefig("arbitrary_payoff_proportion_n="+str(n)+".png")
        if(show):
            plt.show()
    return
#plot_determinist(A)

In [8]:
def solve(T, A, plh, phl, X0 = np.zeros(3) + 1./3):
    dt = T[1] - T[0]
    XX = np.zeros((len(T), 3))
    
    XX[0] = X0
    
    l = False
    h = True
    NN = np.zeros_like(T)
    
    ph = plh/(plh+phl)
    NN[0] = (np.random.binomial(n=1, p=ph)==1)
    for i, t in enumerate(T[:-1:]):
        if(NN[i]==l):
            if(np.random.rand()<plh*dt):
                NN[i+1] = h
                An = A[1]
            else:
                NN[i+1] = l
                An = A[0]
        elif(NN[i]==h):
            if(np.random.rand()<phl*dt):
                NN[i+1] = l
                An = A[0]
            else:
                NN[i+1] = h
                An = A[1]
        XX[i+1] = XX[i] + dt*F(XX[i], An)
    return XX, NN

In [9]:
def plot_nitrogen(T, XX, NN, forest_type):
#    plt.figure(figsize=(10,5))
    plt.plot(T, NN)
    plt.ylabel("Time", fontsize = 25)
    plt.ylabel("Nitrogen", fontsize = 25)
    plt.yticks([0,1], ["low", "high"])
    plt.title(forest_type, fontsize=20)
    #plt.savefig("arbitrary_payoff_"+forest_type+"_nitrogen.png")
    return

In [10]:
def plot_biomass(T, XX, NN, forest_type):
 #   plt.figure(figsize=(10,5))
    for i in range(3):
        plt.plot(T, XX[:,i], label=Species[i])
    #plt.plot(T, XX[:,3], label="x_4")
    #plt.plot(T, np.sum(XX, axis = 1), "--", color = "grey", label="sum")
    #plt.legend(fontsize = 20, loc="lower right")
    plt.xlabel("Time", fontsize = 25)
    plt.ylabel("Biomass proportion", fontsize = 25)
    plt.title(forest_type, fontsize=20)
    #plt.savefig("arbitrary_payoff_"+forest_type+".png")
#    plt.show()

In [11]:
T = np.linspace(0, 4000, 20001)

In [12]:
T = np.linspace(0, 10000, 1001)

In [13]:
Species = ['non fixer', 'facultative fixer', 'obligate fixer']

In [14]:
alpha, beta, gamma, Alpha, Beta, Gamma = [0.001, 0.008, 0.0036, 0.00036, 0.005, 0.0016]

# Dashboard

In [15]:
class Select_box(widgets.VBox):
    def __init__(self, dashboard):
        
        import numpy as np
        
        self.dashboard = dashboard


        #initial point
        self.selection_non0= widgets.FloatSlider(min = 0, max = 1, description = "non fixers t=0", value = 1./3)
        self.selection_non0.observe(dashboard.on_non0_selected, names = "value")
        
        self.selection_fac0= widgets.FloatSlider(min = 0, max = 1, description = "facultative t=0", value = 1./3)
        self.selection_fac0.observe(dashboard.on_fac0_selected, names = "value")
        
        self.selection_obl0= widgets.FloatSlider(min = 0, max = 1, description = "obligate t=0", value = 1./3)
        self.selection_obl0.observe(dashboard.on_obl0_selected, names = "value")
        
        

        self.selection_plh_t = widgets.FloatLogSlider(min = np.log10(0.005), max = np.log10(0.5), description = "plh tropics", value = 0.05)
        self.selection_plh_t.observe(dashboard.on_plh_t_selected, names = "value")
        
        self.selection_phl_t = widgets.FloatLogSlider(min = np.log10(0.002), max = np.log10(0.2), description = "phl tropics", value = 0.02)
        self.selection_phl_t.observe(dashboard.on_phl_t_selected, names = "value")

        self.selection_plh_e = widgets.FloatLogSlider(min = np.log10(0.001), max = np.log10(0.2), description = "plh extra", value = 0.02)
        self.selection_plh_e.observe(dashboard.on_plh_e_selected, names = "value")
        
        self.selection_phl_e = widgets.FloatLogSlider(min = np.log10(0.001), max = np.log10(0.1), description = "phl extra", value = 0.01)
        self.selection_phl_e.observe(dashboard.on_plh_e_selected, names = "value")

        # [0.001, 0.008, 0.0036, 0.00036, 0.005, 0.0016]
        self.selection_alpha = widgets.FloatLogSlider(min = np.log10(0.0001), max = np.log10(0.01), description = "alpha", value = 0.001)
        self.selection_alpha.observe(dashboard.on_alpha_selected, names = "value")
        
        self.selection_beta = widgets.FloatLogSlider(min = np.log10(0.0008), max = np.log10(0.08), description = "beta", value = 0.008)
        self.selection_beta.observe(dashboard.on_beta_selected, names = "value")

        self.selection_gamma = widgets.FloatLogSlider(min = np.log10(0.00036), max = np.log10(0.036), description = "gamma", value = 0.0036)
        self.selection_gamma.observe(dashboard.on_gamma_selected, names = "value")

        self.selection_A = widgets.FloatLogSlider(min = np.log10(0.000036), max = np.log10(0.0036), description = "A", value = 0.00036)
        self.selection_A.observe(dashboard.on_A_selected, names = "value")

        self.selection_B = widgets.FloatLogSlider(min = np.log10(0.0005), max = np.log10(0.05), description = "B", value = 0.005)
        self.selection_B.observe(dashboard.on_B_selected, names = "value")

        self.selection_Gamma = widgets.FloatLogSlider(min = np.log10(0.00016), max = np.log10(0.016), description = "Gamma", value = 0.0016)
        self.selection_Gamma.observe(dashboard.on_Gamma_selected, names = "value")

        
            
        
        
        children = [
        self.selection_non0,
        self.selection_fac0,
        self.selection_obl0,
        self.selection_phl_t,
        self.selection_plh_t,
        self.selection_phl_e,
        self.selection_plh_e,
        self.selection_alpha,
        self.selection_beta,
        self.selection_gamma,
        self.selection_A,
        self.selection_B,
        self.selection_Gamma,
        ]
        super().__init__(children, layout=Layout(width="100%"))   

In [16]:
class Image_box(widgets.Box):
    def __init__(self, dashboard):
        #%pip install matplotlib
        #import matplotlib.pyplot as plt
        self.image = widgets.Image()
        self.dashboard = dashboard
        
        self.non0 = self.dashboard.non0
        self.fac0 = self.dashboard.fac0
        self.obl0 = self.dashboard.obl0
        
        self.phl_t = self.dashboard.phl_t
        self.plh_t = self.dashboard.plh_t
        self.phl_e = self.dashboard.phl_e
        self.plh_e = self.dashboard.plh_e
  
        self.alpha = self.dashboard.alpha
        self.beta = self.dashboard.beta
        self.gamma = self.dashboard.gamma
        self.A = self.dashboard.A
        self.B = self.dashboard.B
        self.Gamma = self.dashboard.Gamma
        
        
        self.print_image()
        
        image_container = widgets.Box([self.image], layout=Layout(width="100%"))
        
        children = [
   #     image_container,
        self.image,   
        ]
        super().__init__(children, layout=Layout(width="100%"))
        
    def print_image(self):
        """
        try:
            import matplotlib.pyplot as plt
        except:
            %pip install matplotlib -qqq
            import matplotlib.pyplot as plt
        """
        
        A = payoff_matrix_param(self.alpha, self.beta, self.gamma, self.A, self.B, self.Gamma)
        XXe, NNe = solve(T, A, plh = self.plh_e, phl = self.phl_e, X0 = [self.non0, self.fac0, self.obl0]) 
        XXt, NNt = solve(T, A, plh = self.plh_t, phl = self.phl_t, X0 = [self.non0, self.fac0, self.obl0]) 
        
       # fig = plt.figure(figsize=(16, 6))
        plt.subplots(2,2, figsize = (10, 4)) 
        plt.subplot(2,2,1)
        plot_biomass(T, XXt, NNt, "Tropical") 
        plt.ylabel("Biomass proportion", fontsize = 12)
        plt.subplot(2,2,2)
        plot_biomass(T, XXe, NNe, "Extra tropical")
        plt.ylabel("")
        #plt.show()
        
#        plt.subplots(1,2, figsize = (20, 1)) 
        plt.subplot(2,2,3)  ; plot_nitrogen(T, XXt, NNt, "Tropical")  ;
        plt.ylabel("Nitrogen", fontsize = 10)
        plt.title("")
        plt.yticks([0,1], ["low", "high"])
        plt.xlabel("Time", fontsize = 12)
        plt.subplot(2,2,4) ; plot_nitrogen(T, XXe, NNe, "Extra tropical")
        plt.title("")
        plt.yticks([0,1], ["low", "high"])
        plt.ylabel("", fontsize = 12)
        plt.xlabel("Time", fontsize = 12)
        #plt.show()    
        
        
        """
        D = Dynamic(h=h, l=l, alpha=alpha, c=c, model=self.model, tFinal=50)
        D.initialisation()
        if(self.plot == "time series"):
            D.eulerEx()
            D.plot(show=False)
            plt.legend(fontsize = 20) # duplicate ...
        elif(self.plot == "phase portrait"):
            D.plot_phase_portrait(show=False)
        elif(self.plot == "histogram"):
            D.plot_histogram(show=False)
        """
            
        image_file = BytesIO()
        #fig.savefig(fname = image_file)
        plt.savefig(fname = image_file)
        image_file.seek(0)
        image_data = image_file.read()
        self.image.value = image_data
#            self.image.width = 1500
#           self.image.height = 2000
        plt.close()

#          file = open(nom, "rb")
#         image = file.read()
        #plt.imshow(image)
#        self.image.value = image
 #       self.image.format = 'png'
        
        
        return
        

    def change_non0(self, change):
        self.non0 = change
        self.print_image()
        return
    
    def change_fac0(self, change):
        self.fac0 = change
        self.print_image()
        return
    
    def change_obl0(self, change):
        self.obl0 = change
        self.print_image()
        return
    
    def change_plh_t(self, change):
        self.plh_t = change
        self.print_image()
        return
    
    
    def change_phl_t(self, change):
        self.phl_t = change
        self.print_image()
        return
    
    def change_plh_e(self, change):
        self.plh_e = change
        self.print_image()
        return
    
    def change_phl_e(self, change):
        self.phl_e = change
        self.print_image()
        return
    
    
    def change_alpha(self, change):
        self.alpha = change
        self.print_image()
        return
    
    def change_beta(self, change):
        self.beta = change
        self.print_image()
        return
    
    def change_gamma(self, change):
        self.gamma = change
        self.print_image()
        return
    
    
    def change_A(self, change):
        self.A = change
        self.print_image()
        return
    
    def change_B(self, change):
        self.B = change
        self.print_image()
        return
    
    def change_Gamma(self, change):
        self.Gamma = change
        self.print_image()
        return

In [19]:
class Dashboard(widgets.VBox):
    def __init__(self):

        self.non0 = 1./3
        self.fac0 = 1./3
        self.obl0 = 1./3
        
        self.phl_t = 0.05
        self.plh_t = 0.02
        self.phl_e = 0.02
        self.plh_e = 0.01
          
        # [0.001, 0.008, 0.0036, 0.00036, 0.005, 0.0016]
        self.alpha = 0.001
        self.beta = 0.008
        self.gamma = 0.0036
        self.A = 0.00036
        self.B = 0.005
        self.Gamma = 0.0016
    
        self.select_box = Select_box(self)
    #    self.text_box = Text_box(self)
        self.image_box = Image_box(self)
        
        
        #C1 = widgets.Box([self.image_box], layout=Layout(width="65%"))
        #C2 = widgets.Box([self.select_box], layout=Layout(width="32%"))
        
        rowA = widgets.Box([self.image_box, self.select_box], layout=Layout(width="100%"))
        #rowA = widgets.VBox([C1, C2], layout=Layout(width="100%"))
        super().__init__([rowA], layout=Layout(width="100%"))
    

    
    
    def on_non0_selected(self, change):
        self.non0 = change["new"]
        self.image_box.change_non0(change["new"])

    def on_fac0_selected(self, change):
        self.fac0 = change["new"]
        self.image_box.change_fac0(change["new"])

    def on_obl0_selected(self, change):
        self.obl0 = change["new"]
        self.image_box.change_obl0(change["new"])

    def on_plh_t_selected(self, change):
        self.plh_t = change["new"]
        self.image_box.change_plh_t(change["new"])

    def on_phl_t_selected(self, change):
        self.phl_t = change["new"]
        self.image_box.change_phl_t(change["new"])

    def on_plh_e_selected(self, change):
        self.plh_e = change["new"]
        self.image_box.change_plh_e(change["new"])

    def on_phl_e_selected(self, change):
        self.phl_e = change["new"]
        self.image_box.change_phl_e(change["new"])

    def on_alpha_selected(self, change):
        self.alpha = change["new"]
        self.image_box.change_alpha(change["new"])

    def on_beta_selected(self, change):
        self.beta = change["new"]
        self.image_box.change_beta(change["new"])

    def on_gamma_selected(self, change):
        self.gamma = change["new"]
        self.image_box.change_gamma(change["new"])

    def on_A_selected(self, change):
        self.A = change["new"]
        self.image_box.change_A(change["new"])

    def on_B_selected(self, change):
        self.B = change["new"]
        self.image_box.change_B(change["new"])

    def on_Gamma_selected(self, change):
        self.Gamma = change["new"]
        self.image_box.change_Gamma(change["new"])

In [20]:
D=Dashboard()
D

Dashboard(children=(Box(children=(Image_box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x0…